In [2]:
import pandas as pd

excel_file = 'eu_enipi.xlsx'

In [7]:

df1 = pd.read_excel(excel_file, sheet_name=0)
df2 = pd.read_excel(excel_file, sheet_name=1)
# df1.head()
df2.head()

,ID,BREED,SIZE,CEPHALIC_INDEX,SEX,INTACT_VS_NEUTERED,MALIGNANCY,AGE
0,65453,Mixed-breed,NaN,NaN,M,I,B,0.6
1,65454,Mixed-breed,NaN,NaN,NaN,NaN,M,12.0
2,65456,Italian Sighthound,Small,Dolichocephalic,M,I,M,11.0
3,65457,Labrador Retriever,Large,Mesocephalic,F,I,M,3.0
4,65459,Mixed-breed,NaN,NaN,F,I,B,11.0


In [18]:
sex_df = df2.groupby('SEX').agg(
    age_mean=('AGE', 'mean')
).reset_index()
sex_df

,SEX,age_mean
0,F,8.997592
1,M,9.062969


In [19]:
malignancy_df = df2.groupby('MALIGNANCY').agg(
    age_mean=('AGE', 'mean')
).reset_index()
malignancy_df

,MALIGNANCY,age_mean
0,B,8.455493
1,M,9.363610


In [22]:
sex_malignancy_df = df2.groupby('SEX').agg(
    total_count=('MALIGNANCY', 'count'),
    m_count=('MALIGNANCY', lambda x: (x == 'M').sum())
).reset_index()
sex_malignancy_df['percentage_m'] = (sex_malignancy_df['m_count'] / sex_malignancy_df['total_count']) * 100

sex_malignancy_df

,SEX,total_count,m_count,percentage_m
0,F,7276,5048,69.378780
1,M,5625,2990,53.155556


In [23]:
breed_df = df2.groupby('BREED').agg(
    total_count=('MALIGNANCY', 'count'),
    m_count=('MALIGNANCY', lambda x: (x == 'M').sum())
).reset_index()
breed_df['%_m'] = (breed_df['m_count'] / breed_df['total_count']) * 100

breed_df

,BREED,total_count,m_count,%_m
0,Afghan Hound,5,3,60.000000
1,Airedale Terrier,10,4,40.000000
2,Akita Inu,18,9,50.000000
3,Alaskan Malamute,13,10,76.923077
4,American Bulldog,2,2,100.000000
...,...,...,...,...
151,West Highland White Terrier,86,41,47.674419
152,Whippet,21,11,52.380952
153,White Swiss Shepherd Dog,3,3,100.000000
154,Yorkshire Terrier,291,180,61.855670


In [25]:
i_n_df = df2.groupby('INTACT_VS_NEUTERED').agg(
    total_count=('MALIGNANCY', 'count'),
    m_count=('MALIGNANCY', lambda x: (x == 'M').sum())
).reset_index()
i_n_df['%_m'] = (i_n_df['m_count'] / i_n_df['total_count']) * 100

i_n_df

,INTACT_VS_NEUTERED,total_count,m_count,%_m
0,I,9698,5848,60.301093
1,N,3212,2196,68.368618


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatment_labels = ['Intact', 'Neutered']

# Create empty list to fill with tumor vol data (for plotting)
tumor_volumes = []
# outliers = []

# Calculate the IQR and quantitatively determine if there are any potential outliers.
for drug in treatment_labels:
    outliers = []
    # Locate the rows which contain mice on each drug and get the tumor volumes
    tumor_vol_data = df2.loc[df2['INTACT_VS_NEUTERED'] == drug, 'AGE'].astype(float).tolist()
    
    # add subset
    tumor_volumes.append(tumor_vol_data)

    # Convert to a Pandas Series
    tumor_vol_series = pd.Series(tumor_vol_data)
    
    # Determine quartiles
    quartiles = tumor_vol_series.quantile([0.25, 0.5, 0.75])
    
    # Determine outliers using upper and lower bounds
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq - lowerq
    
    lower_bound = lowerq - (1.5 * iqr)
    upper_bound = upperq + (1.5 * iqr)
    
    for volume in tumor_vol_data:
        if volume < lower_bound or volume > upper_bound:
            outliers.append(volume)
            
    output = f'''*******************
Drug Reginmen: {drug}
Upper Bound: {upper_bound}
Lower Bound: {lower_bound}
IQR: {iqr}
Outliers: {outliers}
'''
    print(output)


In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Tumor Volume for Drug Regimens')
ax1.set_ylabel('Tumor Volume (mm3)')
flierprops = dict(marker='+', 
                  color='red')
ax1.boxplot(tumor_volumes, 
            labels=treatment_labels, 
            flierprops=flierprops)
plt.show()